Script to Delete Retired Units that are invalid
Update Unit Status from Held to issue, as well as the original unit counts


For context, all the uploaded units that belong to this orgUid are all CDM data. Therefore, we dont need to run another script with v1/projects API to match whose unit the registry belong too

In [10]:
# Reproduce the code
# Get Units
# Get all units for a given organization
# {{hostURL}}/v1/units?page=1&limit=100&orgUid={{orgUid}}

# Loop through all pages and store in dataframe
# Only store it if "unitStatus" is "retired"
# Just nice that all the "retired" units all belong to CDM. No need to have another checking whose unit belong to what registry. Otherwise, proeject level APi checking is needed

In [15]:
import pandas as pd
import requests
import json


In [19]:


# THIS SCRIPT IS TO RUN ONCE ONLY, THEN SAVED THE UNITS TO CSV FILE. FOR FURTHER ETL PROCESSING, USE THE CSV FILE SO NO MORE API CALLS ARE NEEDED


# Define the base URL and organization UID
base_url = "http://localhost:31310/v1/units"
org_uid = "b3d4e71d806e86ff1f8712b6854d65e2c178e873ee22b2f7d0da937dacbaa985"

# Initialize variables
page = 1
limit = 100
all_units = []
# Loop through all pages
while True:
    # Construct the URL with pagination and organization UID
    url = f"{base_url}?page={page}&limit={limit}&orgUid={org_uid}"
    
    # Make the GET request
    response = requests.get(url)
    
    if response.status_code != 200:
        print(f"Failed to fetch data: {response.status_code}")
        break
    
    # Parse the response JSON
    data = response.json()
    
    # Extract units from the response
    units = data.get("data", [])
    
    # Filter for units with "unitStatus" == "Retired"
    retired_units = [unit for unit in units if unit.get("unitStatus") == "Retired"]
    all_units.extend(retired_units)
    
    # Check if there are more pages
    if page >= data.get("pageCount", 0):
        break
    
    # Increment the page number
    page += 1

# Convert the list of retired units to a DataFrame
df_held_units = pd.DataFrame(all_units)

# Save the DataFrame to a CSV file for further analysis
df_held_units.to_csv("undeletedUnits.csv", index=False)

print(f"Fetched {len(df_held_units)} retired units.")

Fetched 271 retired units.


In [20]:
df_held_units

# ...existing code...

df_held_units_cdm = df_held_units[df_held_units['unitRegistryLink'].str.startswith("https://www.energyabsolute.co.th")]

print(f"Filtered to {len(df_held_units_cdm)} CDM units.")
df_held_units_cdm
# ...existing code...

Filtered to 0 CDM units.


,warehouseUnitId,issuanceId,projectLocationId,orgUid,unitOwner,countryJurisdictionOfOwner,inCountryJurisdictionOfOwner,serialNumberBlock,unitBlockStart,unitBlockEnd,...,unitStatus,unitStatusReason,unitRegistryLink,correspondingAdjustmentDeclaration,correspondingAdjustmentStatus,timeStaged,createdAt,updatedAt,labels,issuance


In [22]:
df_held_units_cdm = pd.read_csv("undeletedUnits.csv")

df_held_units_cdm_ids = df_held_units_cdm['warehouseUnitId']



def delete_retired_units(df_retired_units, base_url, start_index=0, end_index=None):
    """
    Deletes retired units based on their warehouseUnitId.

    Parameters:
        df_retired_units (DataFrame): DataFrame containing retired units.
        base_url (str): The base URL for the DELETE API.
        start_index (int): The starting index for deletion.
        end_index (int): The ending index for deletion. If None, deletes until the end of the DataFrame.
    """
    # Get the list of warehouseUnitIds
    df_held_units_cdm_ids = df_retired_units['warehouseUnitId'][start_index:end_index]

    # Loop through each retired unit ID and send a DELETE request
    for idx,unit_id in enumerate(df_held_units_cdm_ids):
        # Construct the request body
        request_body = {
            "warehouseUnitId": unit_id
        }
        
        # Send the DELETE request
        response = requests.delete(base_url, json=request_body)
        
        # Check the response status
        if response.status_code == 200:
            print(f"Successfully deleted unit entry number {start_index+idx} with ID: {unit_id}")
        else:
            print(f"Failed to delete unit with ID: {unit_id}. Status code: {response.status_code}. Message: {response.text}")

# Example usage
base_url = "http://localhost:31310/v1/units"
start_index = 100  # Specify the starting index
end_index = 300  # Specify the ending index (or None to delete all remaining units)

delete_retired_units(df_held_units_cdm, base_url, start_index, end_index)

Successfully deleted unit entry number 100 with ID: a0f87ddf-d2ed-4a2b-bf0f-2587ed06a8ec
Successfully deleted unit entry number 101 with ID: e745e430-f7e5-4543-9101-e7d73846b477
Successfully deleted unit entry number 102 with ID: 848f82f8-1da6-4e03-909b-85da485ef86b
Successfully deleted unit entry number 103 with ID: c3b48ac7-ed2d-428a-b38b-78c145f2d9cb
Successfully deleted unit entry number 104 with ID: 255099d1-6908-4c98-a57b-f6d1d602dbe4
Successfully deleted unit entry number 105 with ID: 2079bcab-6c99-4ebe-8ef9-20bfe51257e4
Successfully deleted unit entry number 106 with ID: 3b9c5d89-f9d2-43e9-b079-238d4e687d04
Successfully deleted unit entry number 107 with ID: cb867191-7456-4534-b25d-a62e18313d46
Successfully deleted unit entry number 108 with ID: 66f1f70b-b503-4375-8276-60062748b20e
Successfully deleted unit entry number 109 with ID: 0f45db7e-f1b5-4289-a8b3-0e3b407dc8a4
Successfully deleted unit entry number 110 with ID: 72fe2487-f196-4bdd-b401-ed5b321ea095
Successfully deleted 